In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
# If you need fully deterministic results between runs, set the following environment value prior to launching jupyter.
# See comment in sherlock.features.paragraph_vectors.infer_paragraph_embeddings_features for more info.
%env PYTHONHASHSEED

'13'

In [3]:
from datetime import datetime

print(f'Started at {datetime.now()}.')

Started at 2022-02-04 08:08:02.283035.


# Extract features, retrain Sherlock and generate predictions.

The script below first downloads the data (roughly 700K samples), then extract features from the raw data values. <br>
If you want to skip this step, you can follow the steps below the feature extraction to load the preprocessed data, 
retrain Sherlock and generate predictions.

In [4]:
import sys

import numpy as np
import pandas as pd

from sherlock import helpers
from sherlock.features.preprocessing import extract_features, convert_string_lists_to_lists, prepare_feature_extraction

## Download data
This will download the raw values and preprocessed files, the corresponding labels as well as a few other supporting files:
- `download_data()` will download 3.6GB of data into the `data/` directory.
- `prepare_feature_extraction()` will download +/- 800 MB of data into the `features/` directory.

In [5]:
helpers.download_data()
prepare_feature_extraction()

Data was downloaded.
Preparing feature extraction by downloading 2 files:
        
 ../sherlock/features/glove.6B.50d.txt and 
 ../sherlock/features/par_vec_trained_400.pkl.docvecs.vectors_docs.npy.
        
All files for extracting word and paragraph embeddings are present.


## Read in raw data
You can skip this step if you want to use a preprocessed data file.

## Report memory usage (can be slow)

In [6]:
report_memory = False

## Extract features
It is important that the string-representations of lists are first converted into lists of strings.
The labels should be a list of semantic types.

In [7]:
import time 

timestr = time.strftime("%Y%m%d-%H%M%S")

# Features will be output to the following files
X_test_filename_csv = f'../data/work/test_{timestr}.csv'
X_train_filename_csv = f'../data/work/train_{timestr}.csv'
X_validation_filename_csv = f'../data/work/validation_{timestr}.csv'

### PREPARATION

In [8]:
# ensure embedding initialisation is outside of timing for extract_features
from sherlock.features.word_embeddings import initialise_word_embeddings
from sherlock.features.paragraph_vectors import initialise_pretrained_model, initialise_nltk
from sherlock.features.preprocessing import prepare_feature_extraction

prepare_feature_extraction()
initialise_word_embeddings()
initialise_pretrained_model(400)
initialise_nltk()

Preparing feature extraction by downloading 2 files:
        
 ../sherlock/features/glove.6B.50d.txt and 
 ../sherlock/features/par_vec_trained_400.pkl.docvecs.vectors_docs.npy.
        
All files for extracting word and paragraph embeddings are present.
Initialising word embeddings
Initialise Word Embeddings process took 0:00:06.272654 seconds.
Initialise Doc2Vec Model, 400 dim, process took 0:00:03.008532 seconds. (filename = ../sherlock/features/par_vec_retrained_400.pkl)
Initialised NLTK, process took 0:00:00.189256 seconds.


[nltk_data] Downloading package punkt to /Users/lowecg/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/lowecg/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [9]:
from pyarrow.parquet import ParquetFile

def load_parquet_values(path):
    pf = ParquetFile(source=path)
    row_df = pf.read_row_group(0)

    return row_df['values']

In [10]:
%load_ext line_profiler

In [11]:
import multiprocessing as mp

# default process creation mode is changed in Python 3.8 to 'spawn' which causes "name not defined" type errors. Force fork 'mode' for now (this only needs to be called once).
# https://bugs.python.org/issue39931
#mp.set_start_method('fork', force=True)

## EXTRACT FEATURES TO CSV

### TEST SET

In [12]:
from sherlock.functional import extract_features_to_csv

values = load_parquet_values("../data/raw/test_values.parquet")

extract_features_to_csv(X_test_filename_csv, values)

values = None

Starting ../data/work/test_20220204-080804.csv at 2022-02-04 08:08:15.084360. Rows=137353, using 8 CPU cores
Exporting 1588 column features
Finished. Processed 137353 rows in 0:04:41.968998, key_count=8


In [13]:
print(f'Finished at {datetime.now()}')

Finished at 2022-02-04 08:12:57.143206


### TRAIN SET

In [14]:
values = load_parquet_values("../data/raw/train_values.parquet")

extract_features_to_csv(X_train_filename_csv, values)

values = None

Starting ../data/work/train_20220204-080804.csv at 2022-02-04 08:12:59.783668. Rows=412059, using 8 CPU cores
Exporting 1588 column features
Finished. Processed 412059 rows in 0:13:58.582135, key_count=8


In [15]:
print(f'Finished at {datetime.now()}')

Finished at 2022-02-04 08:26:58.568907


### VALIDATION SET

In [16]:
values = load_parquet_values("../data/raw/val_values.parquet")

extract_features_to_csv(X_validation_filename_csv, values)

values = None

Starting ../data/work/validation_20220204-080804.csv at 2022-02-04 08:26:59.404661. Rows=137353, using 8 CPU cores
Exporting 1588 column features
Finished. Processed 137353 rows in 0:04:39.179751, key_count=8


In [17]:
print(f'Finished at {datetime.now()}')

Finished at 2022-02-04 08:31:38.661961


### Read Locally Processed Features

In [18]:
start = datetime.now()

X_test = pd.read_csv(X_test_filename_csv, dtype=np.float32)

print(f'Load Features (test) process took {datetime.now() - start} seconds.')

Load Features (test) process took 0:00:32.595395 seconds.


In [19]:
X_test.head()

,n_[0]-agg-any,n_[0]-agg-all,n_[0]-agg-mean,n_[0]-agg-var,n_[0]-agg-min,n_[0]-agg-max,n_[0]-agg-median,n_[0]-agg-sum,n_[0]-agg-kurtosis,n_[0]-agg-skewness,...,par_vec_390,par_vec_391,par_vec_392,par_vec_393,par_vec_394,par_vec_395,par_vec_396,par_vec_397,par_vec_398,par_vec_399
0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,-3.000000,0.000000,...,0.030798,0.039629,0.050936,-0.000418,0.110443,0.083834,-0.006636,-0.098172,0.057118,0.024585
1,1.0,0.0,0.368421,0.337950,0.0,2.0,0.0,7.0,0.742677,1.326868,...,0.017153,-0.117459,-0.052406,-0.026878,0.107285,0.057874,-0.045235,-0.079498,0.091608,0.179564
2,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,-3.000000,0.000000,...,-0.197937,-0.341938,-0.195952,0.038289,0.038866,0.113934,0.027655,-0.116132,-0.064190,-0.148824
3,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,-3.000000,0.000000,...,0.090470,-0.063266,0.002614,-0.026875,-0.018085,0.060270,0.042334,-0.200745,0.132219,0.025028
4,1.0,0.0,0.020080,0.035741,0.0,2.0,0.0,5.0,96.521561,9.784149,...,0.334674,-0.122423,-0.373649,-0.090392,-0.165679,0.238570,0.504209,-0.262324,-0.041570,-0.468184


In [20]:
start = datetime.now()

X_train = pd.read_csv(X_train_filename_csv, dtype=np.float32)

print(f'Load Features (train) process took {datetime.now() - start} seconds.')

Load Features (train) process took 0:01:42.386525 seconds.


In [21]:
X_train.head()

,n_[0]-agg-any,n_[0]-agg-all,n_[0]-agg-mean,n_[0]-agg-var,n_[0]-agg-min,n_[0]-agg-max,n_[0]-agg-median,n_[0]-agg-sum,n_[0]-agg-kurtosis,n_[0]-agg-skewness,...,par_vec_390,par_vec_391,par_vec_392,par_vec_393,par_vec_394,par_vec_395,par_vec_396,par_vec_397,par_vec_398,par_vec_399
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-3.0,0.0,...,-0.001600,-0.025123,0.068435,0.028728,0.123009,0.055431,0.056169,-0.075245,0.068694,0.135792
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-3.0,0.0,...,0.047790,-0.074355,-0.000537,0.029701,0.171086,0.024220,0.051500,-0.021072,0.147295,0.157399
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-3.0,0.0,...,-0.111040,-0.262439,0.294320,-0.233797,-0.049520,0.084176,-0.124577,-0.300694,0.024991,0.179304
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-3.0,0.0,...,-0.023044,-0.228889,0.028576,0.026734,0.100288,0.014854,0.013547,-0.009963,0.083362,0.208647
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-3.0,0.0,...,0.241473,-0.244866,0.155266,0.239482,0.181161,-0.006200,0.094836,-0.216232,0.332968,0.203908


In [22]:
start = datetime.now()

X_validation = pd.read_csv(X_validation_filename_csv, dtype=np.float32)

print(f'Load Features (validation) process took {datetime.now() - start} seconds.')

Load Features (validation) process took 0:00:32.728528 seconds.


In [23]:
X_validation.head()

,n_[0]-agg-any,n_[0]-agg-all,n_[0]-agg-mean,n_[0]-agg-var,n_[0]-agg-min,n_[0]-agg-max,n_[0]-agg-median,n_[0]-agg-sum,n_[0]-agg-kurtosis,n_[0]-agg-skewness,...,par_vec_390,par_vec_391,par_vec_392,par_vec_393,par_vec_394,par_vec_395,par_vec_396,par_vec_397,par_vec_398,par_vec_399
0,0.0,0.0,0.00000,0.000000,0.0,0.0,0.0,0.0,-3.000000,0.000000,...,-0.055852,-0.309699,0.052984,-0.230841,0.276387,-0.327680,0.028990,-0.345448,0.059717,-0.009583
1,0.0,0.0,0.00000,0.000000,0.0,0.0,0.0,0.0,-3.000000,0.000000,...,0.157757,-0.241466,0.110374,0.100078,0.226504,-0.081568,0.067137,-0.399932,-0.002087,0.224772
2,1.0,0.0,0.25000,0.187500,0.0,1.0,0.0,10.0,-0.666667,1.154701,...,0.315950,0.139812,0.134535,-0.180131,0.317000,0.061368,0.427069,-0.508263,-0.049458,0.104446
3,0.0,0.0,0.00000,0.000000,0.0,0.0,0.0,0.0,-3.000000,0.000000,...,-0.031292,-0.266281,-0.065231,0.024238,0.132858,0.138291,0.138538,-0.004716,0.117068,-0.083835
4,1.0,0.0,0.09434,0.311855,0.0,4.0,0.0,5.0,42.275021,6.530585,...,0.061921,-0.222074,-0.254157,0.251418,0.429034,-0.411838,-0.092745,-0.182875,0.076406,-0.079038


## Impute NaN values with feature means

In [24]:
start = datetime.now()

train_columns_means = pd.DataFrame(X_train.mean()).transpose()

print(f'Transpose process took {datetime.now() - start} seconds.')

Transpose process took 0:00:20.667815 seconds.


In [25]:
start = datetime.now()

X_train.fillna(train_columns_means.iloc[0], inplace=True)
X_validation.fillna(train_columns_means.iloc[0], inplace=True)
X_test.fillna(train_columns_means.iloc[0], inplace=True)

train_columns_means=None

print(f'FillNA process took {datetime.now() - start} seconds.')

FillNA process took 0:00:02.789372 seconds.


In [26]:
start = datetime.now()

X_train.to_parquet('train.parquet', engine='pyarrow', compression='snappy')
X_validation.to_parquet('validation.parquet', engine='pyarrow', compression='snappy')
X_test.to_parquet('test.parquet', engine='pyarrow', compression='snappy')

print(f'Save parquet process took {datetime.now() - start} seconds.')

Save parquet process took 0:00:59.647533 seconds.


In [27]:
print(f'Completed at {datetime.now()}.')

Completed at 2022-02-04 08:35:50.087401.
